In [1]:
import ollama
import os
import json
from tqdm import tqdm

In [3]:
ollama.list()

{'models': [{'name': 'mistral:7b',
   'model': 'mistral:7b',
   'modified_at': '2024-05-25T17:17:04.703920594+02:00',
   'size': 4113301090,
   'digest': '2ae6f6dd7a3dd734790bbbf58b8909a606e0e7e97e94b7604e0aa7ae4490e6d8',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '7.2B',
    'quantization_level': 'Q4_0'},
   'expires_at': '0001-01-01T00:00:00Z'},
  {'name': 'llama3:latest',
   'model': 'llama3:latest',
   'modified_at': '2024-05-24T15:13:40.464991529+02:00',
   'size': 4661224676,
   'digest': '365c0bd3c000a25d28ddbf732fe1c6add414de7275464c4e4d1c3b5fcb5d8ad1',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'Q4_0'},
   'expires_at': '0001-01-01T00:00:00Z'}]}

In [4]:
# Function to get response from ollama.chat
def get_response(json_content):
    response = ollama.chat(model='mistral:7b', messages=[
        {
            'role': 'system',
            'content': f"""
<s>[INST]
Please extract the following features from the given ECHR case information. These features will be used later for classification purposes:

1. **Length of Legal Reasoning Section:** Provide the word count or a detailed estimate of the length of the legal reasoning section.
2. **Number and Depth of Citations:** List the number of citations to other cases, statutes, or legal principles, and describe the depth of discussion for these citations.
3. **Presence of Legal Principles and Detailed Interpretations:** Identify and describe the legal principles discussed in the case and provide details on how thoroughly they are interpreted and applied.
4. **Multiple Concurring or Dissenting Opinions:** Indicate whether there are any concurring or dissenting opinions, and provide details about their number and depth.
5. **References to the Development or Clarification of Legal Principles:** Note any instances where the case contributes to the development or clarification of existing legal doctrines, and provide details.
6. **Number of Times Subsequent Cases Cite the Case:** If available, provide the number of times this case is cited in subsequent judicial decisions.

### Input:  
Case:
{json_content}
Based on the above information, extract the following features:

1. Length of Legal Reasoning Section:
2. Number and Depth of Citations:
3. Presence of Legal Principles and Detailed Interpretations:
4. Multiple Concurring or Dissenting Opinions:
5. References to the Development or Clarification of Legal Principles:
6. Number of Times Subsequent Cases Cite the Case:

### Response:  

[/INST]
"""
        },
    ])
    return response['message']['content']

In [5]:
# Directory paths
input_dir = './echr-prompts/'
output_dir = './zero-results/'

# Ensure directory exists
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


In [6]:

# Process each JSON file
for filename in tqdm(os.listdir(input_dir)):
    if filename.endswith('.json'):
        file_path = os.path.join(input_dir, filename)
        
        # Load JSON file
        with open(file_path, 'r') as file:
            json_data = json.load(file)

        label = json_data['importance']
        del json_data['importance']
        
        # Convert JSON data to string
        json_string = json.dumps(json_data)
        
        # Get response
        response_content = get_response(json_string)
        
        # Save response to output directory as a .txt file
        output_file_path = os.path.join(output_dir, f'response_{os.path.splitext(filename)[0]}.txt')
        with open(output_file_path, 'w') as output_file:
            output_file.write(label + '-' + response_content)
    break
        


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:39<?, ?it/s]


In [7]:
print(response_content)

 1. Length of Legal Reasoning Section: The length of the legal reasoning section can be estimated as approximately 250 words, based on a standard word count. However, this is an approximation as the structure and density of text may affect the precise word count.
2. Number and Depth of Citations: There are four citations to articles in the European Convention on Human Rights (ECHR). The depth of discussion for these citations appears brief, with each citation being mentioned only once in the conclusion to determine whether a violation has occurred or not.
3. Presence of Legal Principles and Detailed Interpretations: The case discusses several legal principles, including Articles 6, 13, P1-1 (Protocol No. 1), and Article 41 (just satisfaction). However, the interpretation and application of these principles are not provided in detail; instead, the court quickly determines whether a violation has occurred based on these articles.
4. Multiple Concurring or Dissenting Opinions: None is pre

In [11]:
label

'3'